# 주요 영역별 회의 음성인식 데이터 구축
- AI hub의 해당 데이터셋을 준비하는 과정이에요.
- 파인튜닝 직전 단계의 전처리 데이터셋을 구축해서
- 허깅페이스에 업로드 하는 것 까지가 목표에요.

    1. 오디오(mp3), 텍스트(txt) 파일 매핑
    2. 괄호 전처리
    3. 16khz 전처리
    4. 허깅페이스 업로드

In [9]:
import os
import json
from pydub import AudioSegment
from tqdm import tqdm


'''
데이터셋 경로를 지정해서
하나의 폴더에 mp3, txt 파일로 추출하는 코드에요.
추출 과정에서 원본 파일은 자동으로 삭제돼요. (저장공간 절약을 위해)
'''

# 파일 경로 설정
DATA_DIR = '/mnt/a/maxseats-git/New_Sample'


json_base_dir = os.path.join(DATA_DIR, '라벨링데이터')
audio_base_dir = os.path.join(DATA_DIR, '원천데이터')
output_dir = '/mnt/a/maxseats-git/주요 영역별 회의 음성인식 데이터'



def process_audio_and_subtitle(json_path, audio_base_dir, output_dir):
    # JSON 파일 읽기
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 메타데이터에서 오디오 파일 이름 추출
    title = data['metadata']['title']
    
    # 각 TS, VS 폴더에서 해당 오디오 파일을 찾기
    audio_file = None
    for root, _, files in os.walk(audio_base_dir):
        for file in files:
            if file == title + '.wav':
                audio_file = os.path.join(root, file)
                break
        if audio_file:
            break
    
    # 오디오 파일 로드
    if not audio_file or not os.path.exists(audio_file):
        print(f"Audio file {audio_file} does not exist.")
        return
    
    audio = AudioSegment.from_wav(audio_file)
    
    # 발화 데이터 처리
    for utterance in data['utterance']:
        start_time = float(utterance['start']) * 1000  # 밀리초로 변환
        end_time = float(utterance['end']) * 1000      # 밀리초로 변환
        text = utterance['form']
        
        # 오디오 클립 추출
        audio_clip = audio[start_time:end_time]
        
        # 파일 이름 설정
        clip_id = utterance['id']
        audio_output_path = os.path.join(output_dir, clip_id + '.mp3')
        text_output_path = os.path.join(output_dir, clip_id + '.txt')
        
        # 오디오 클립 저장
        audio_clip.export(audio_output_path, format='mp3')
        
        # 텍스트 파일 저장
        with open(text_output_path, 'w', encoding='utf-8') as f:
            f.write(text)

    # 오디오 파일 삭제
    os.remove(audio_file)
    os.remove(audio_file.replace('.wav', '.txt'))
    print(f"Deleted audio file: {audio_file}")

def process_all_files(json_base_dir, audio_base_dir, output_dir):
    json_files = []
    
    # JSON 파일 목록 생성
    for root, dirs, files in os.walk(json_base_dir):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    
    # JSON 파일 처리
    for json_file in tqdm(json_files, desc="Processing JSON files"):
        process_audio_and_subtitle(json_file, audio_base_dir, output_dir)
        
        # 완료 후 JSON 파일 삭제
        os.remove(json_file)
        print(f"Deleted JSON file: {json_file}")

# 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 프로세스 실행
process_all_files(json_base_dir, audio_base_dir, output_dir)


Processing JSON files:  25%|██▌       | 1/4 [01:28<04:25, 88.62s/it]

Deleted audio file: /mnt/a/maxseats-git/New_Sample/원천데이터/TS1/공중파방송/의료/DGBAH21000196.wav
Deleted JSON file: /mnt/a/maxseats-git/New_Sample/라벨링데이터/TL1/공중파방송/의료/DGBAH21000196.json


Processing JSON files:  50%|█████     | 2/4 [04:16<04:29, 134.97s/it]

Deleted audio file: /mnt/a/maxseats-git/New_Sample/원천데이터/TS1/공중파방송/의료/DGBAH21000195.wav
Deleted JSON file: /mnt/a/maxseats-git/New_Sample/라벨링데이터/TL1/공중파방송/의료/DGBAH21000195.json


Processing JSON files:  75%|███████▌  | 3/4 [05:47<01:55, 115.18s/it]

Deleted audio file: /mnt/a/maxseats-git/New_Sample/원천데이터/TS1/공중파방송/의료/DGBAH21000201.wav
Deleted JSON file: /mnt/a/maxseats-git/New_Sample/라벨링데이터/TL1/공중파방송/의료/DGBAH21000201.json


Processing JSON files: 100%|██████████| 4/4 [08:10<00:00, 122.54s/it]

Deleted audio file: /mnt/a/maxseats-git/New_Sample/원천데이터/TS1/공중파방송/의료/DGBAH21000190.wav
Deleted JSON file: /mnt/a/maxseats-git/New_Sample/라벨링데이터/TL1/공중파방송/의료/DGBAH21000190.json


In [ ]:
'''
프롬프트


C:.
│  New_Sample.zip
│
└─New_Sample
    ├─라벨링데이터
    │  └─TL1
    │      └─공중파방송
    │          └─의료
    │                  DGBAH21000190.json
    │                  DGBAH21000195.json
    │                  DGBAH21000196.json
    │                  DGBAH21000201.json
    │
    └─원천데이터
        └─TS1
            └─공중파방송
                └─의료
                        DGBAH21000190.txt
                        DGBAH21000190.wav
                        DGBAH21000195.txt
                        DGBAH21000195.wav
                        DGBAH21000196.txt
                        DGBAH21000196.wav
                        DGBAH21000201.txt
                        DGBAH21000201.wav

파일 구조가 위와 같아. 그리고 json 파일 하나의 예시는 다음과 같아.

{
	"metadata": {
		"title": "DGBAH21000190",
		"creator": "솔트룩스",
		"distributor": "솔트룩스",
		"year": 2021,
		"category": "구어 > 공적 > 회의",
		"date": "20020406",
		"media": "공중파방송",
		"communication": "대면",
		"type": "토론",
		"domain": "의료",
		"topic": "마약의 심각성과 폐해",
		"speaker_num": 17,
		"organization": "",
		"annotation_level": "원시",
		"sampling": "본문 전체"
	},
	"speaker": [
		{
			"id": "DG0001",
			"name": "DG0001",
			"age": "30대",
			"occupation": "성우",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0002",
			"name": "DG0002",
			"age": "30대",
			"occupation": "성우",
			"role": "토론자",
			"sex": "여"
		},
		{
			"id": "DG0003",
			"name": "DG0003",
			"age": "40대",
			"occupation": "성우",
			"role": "사회자",
			"sex": "남"
		},
		{
			"id": "DG0004",
			"name": "DG0004",
			"age": "30대",
			"occupation": "성우",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0005",
			"name": "DG0005",
			"age": "30대",
			"occupation": "성우",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0006",
			"name": "DG0006",
			"age": "40대",
			"occupation": "교수",
			"role": "사회자",
			"sex": "남"
		},
		{
			"id": "DG0007",
			"name": "DG0007",
			"age": "30대",
			"occupation": "아나운서",
			"role": "토론자",
			"sex": "여"
		},
		{
			"id": "DG0008",
			"name": "DG0008",
			"age": "40대",
			"occupation": "아나운서",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0009",
			"name": "DG0009",
			"age": "40대",
			"occupation": "시민",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0010",
			"name": "DG0010",
			"age": "40대",
			"occupation": "의료부장",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0011",
			"name": "DG0011",
			"age": "50대",
			"occupation": "의료전문 변호사",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0012",
			"name": "DG0012",
			"age": "50대",
			"occupation": "교수",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0013",
			"name": "DG0013",
			"age": "50대",
			"occupation": "박사",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0014",
			"name": "DG0014",
			"age": "40대",
			"occupation": "지역 실무회 회장",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0015",
			"name": "DG0015",
			"age": "50대",
			"occupation": "전도사",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0016",
			"name": "DG0016",
			"age": "50대",
			"occupation": "회사원",
			"role": "토론자",
			"sex": "남"
		},
		{
			"id": "DG0017",
			"name": "DG0017",
			"age": "50대",
			"occupation": "시민",
			"role": "토론자",
			"sex": "남"
		}
	],
	"setting": {
		"relation": "사회자 1인 외 토론자"
	},
	"utterance": [
		{
			"id": "DGBAH21000190.1.1.1",
			"start": "0.000",
			"end": "70.699",
			"speaker_id": "DG0001",
			"speaker_role": "토론자",
			"form": "(())",
			"original_form": "(())",
			"environment": "",
			"isIdiom": false,
			"hangeulToEnglish": null,
			"hangeulToNumber": null,
			"term": null
		},
		{
			"id": "DGBAH21000190.1.1.2",
			"start": "70.699",
			"end": "76.904",
			"speaker_id": "DG0001",
			"speaker_role": "토론자",
			"form": "멀티펜으로 바꿨습니다 동호회 아이콘으로 되니까 단번에 갑니다.",
			"original_form": "멀티펜으로 바꿨습니다 동호회 아이콘으로 되니까 단번에 갑니다.",
			"environment": "",
			"isIdiom": false,
			"hangeulToEnglish": null,
			"hangeulToNumber": null,
			"term": null
		},
		{
			"id": "DGBAH21000190.1.1.3",
			"start": "76.904",
			"end": "78.877",
			"speaker_id": "DG0002",
			"speaker_role": "토론자",
			"form": "멀티펜 아이콘으로 단번에",
			"original_form": "멀티펜 아이콘으로 단번에",
			"environment": "",
			"isIdiom": false,
			"hangeulToEnglish": null,
			"hangeulToNumber": null,
			"term": null
		}, ...

이렇게 쭉 이어지는데, 여기서 title로 오디오 파일에 접근해. 그리고 start, end를 통해  해당 시간대를 잘라서 오디오 파일을 mp3 파일로 만들어줘. 또한, form을 참고해서 해당 오디오의 자막을 txt 파일로 만들어줘. 그 두 파일은 확장자만 다르고 이름은 같아야 해.

'''